# Project

https://www.kaggle.com/c/kktv-data-game-1711

Original Training Data: (92951083, 12)


internet_connection_type (7, object): [cellular, cellular 2G, cellular 3G, cellular 4G, offline, online but unknown, wifi]
<br>
platform (3, object): [Android, Web, iOS]
<br>

action_trigger (14, object): [api error, cast, error, interrupt, ..., program stopped, program stopped or enlarged-reduced, seek, video ended]

<br>
最後一個training data的使用者user ID為57158  (data-045.csv)
最後一個label data的使用者user ID為57158  (label-045.csv)


/home/user/data/ML_SVN/DataScience/project

In [2]:
import sys  # handle command argument
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing   #for normalization
from sklearn.model_selection import cross_val_score  # for cross validation
from sklearn import linear_model
from sklearn import  metrics # confusion matrix
from sklearn.model_selection import train_test_split
import itertools #  for  plot_confusion_matrix
import pandas as pd
import os, sys,glob
from pathlib import Path
from datetime import datetime,date
    

In [3]:
def process_time(stime):
    #'2017-11-24 7:59:25.334'
    time_slot=-1
    date1,time1= stime.split(' ')
    date1=[int(i) for i in date1.split('-')]   #[2017, 6, 8] Y/M/D
    time1=[float(i) for i in time1.split(':')]  #[15.0, 18.0, 25.334] H/M/S
    t=datetime(date1[0], date1[1],date1[2],int(time1[0]),int(time1[1])).time()

    t0=t.replace(hour=1,minute=0)
    t1=t.replace(hour=9,minute=0)
    t2=t.replace(hour=17,minute=0)
    t3=t.replace(hour=21,minute=0)
    #print(t)
    #print(t0)  #01:00:00

    if (t>=t0 and t<t1): #AM1:00~AM9:00
        time_slot=0
    elif (t>=t1 and t<t2):  #AM9:00~PM17:00
        time_slot=1
    elif (t>=t2 and t<t3): #PM17:00~PM9:00 
        time_slot=2
    else:  # >PM 9:00
        time_slot=3
    # weekday
    weekday=date(date1[0], date1[1],date1[2]).weekday()   #Modnday:0, Tuesday:1 ,...Sunday:6
    
    return weekday,time_slot




In [11]:
# load data

def read_csv(files):
    #column filter
    
    '''
    all column list:
    user_id,device_id,session_id,title_id,event_time,played_duration,
    action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
    '''
    #olny use partial column
    cols=['user_id','title_id','event_time','played_duration','platform','episode_number','series_total_episodes_count'
         ,'internet_connection_type','is_trailer']
    df=pd.read_csv(files,usecols=cols)
    return df


def preprocess_Xdata():
    print("concatenate all files")
    path = "public/" 
    allFiles=[]
    total_files=75
    for i in range(total_files):
        buf = "data-%03d.csv"%(i+1)
        buf=os.path.join(path,buf)
        allFiles.append(buf)
    print(allFiles)
    df = pd.concat(map(read_csv,allFiles))
    print(df.shape)

    print("pre-processing data")
    df = pd.get_dummies(df, columns=['internet_connection_type'])
    df = pd.get_dummies(df, columns=['platform']) 
    #df = pd.get_dummies(df, columns=['action_trigger']) 
    
    df['event_weekday'], df['event_time_slot']=zip(*df['event_time'].map(process_time))
    df['is_trailer']=df['is_trailer']*1  # change to integer 0 or 1
    #print(df[0:10]['event_time'])
 
    #df.drop(columns=['action_trigger'],inplace=True)
    df.drop(columns=['event_time'],inplace=True)
    #save file
    print("save file")

    df.to_csv('alldata.csv', encoding='utf-8', index=False)
    print("save file Successfully!")

def preprocess_Ydata():
    print("concatenate all files")
    path = "public/" 
    allFiles=[]
    total_files=45
    #allFiles = glob.glob(os.path.join(path,"label-*.csv"))
    for i in range(total_files):
        buf = "label-%03d.csv"%(i+1)
        buf=os.path.join(path,buf)
        allFiles.append(buf)
        
    print(allFiles)
    df = pd.concat(map(pd.read_csv,allFiles))
    print(df.shape)
    print("save file")
    df.to_csv('all_label.csv', encoding='utf-8', index=False)
    print("save file all_label Successfully!")


def load_data(train_data_path,test_data_path):
    
    Train_Data = pd.read_csv(train_data_path, sep=',', header=None)
    Train_Data = np.array(Train_Data.values)
    
    X_train=Train_Data[:,:Train_Data.shape[1]-1]  # 0~(last column-1) are features
    Y_train=Train_Data[:,Train_Data.shape[1]-1]   # the last column is Y (target)

    #X_test = pd.read_csv(test_data_path, sep=',', header=None)
    #X_test = np.array(X_test.values)
    #print(X_test.shape)

    return (X_train, Y_train, X_test)

#write data
def save_data(data,output_file):
    print('write file....')
    path = Path(output_file)
    with open(output_file, 'w') as f:
        for i in data:
            #print(int(item[1]))  #[1] the probability of Spam. 
            f.write('%d\n' %i)
    return





In [12]:
def main():

   
    preprocess_Xdata()
    #preprocess_Ydata()
    # Load feature and label
    #X_all, Y_all, X_test=load_data('alldata.csv')
    #do_PCA(X_all,Y_all)

    print('X_all:',X_all.shape)
    print('X_test:',X_test.shape)
    #Normalization
    X_all =  preprocessing.scale(X_all)
    X_test = preprocessing.scale(X_test)
    #train & validation & test
    #Xtrain, Xvalid, ytrain, yvalid =train_test_split(X_all,Y_all, test_size = 0.1)


    #y=options[method](Xtrain,Xvalid,ytrain,yvalid,X_test)
    #save_data(y,'predict.csv')

    print('Finished !')


main()

concatenate all files
['public/data-001.csv', 'public/data-002.csv', 'public/data-003.csv', 'public/data-004.csv', 'public/data-005.csv', 'public/data-006.csv', 'public/data-007.csv', 'public/data-008.csv', 'public/data-009.csv', 'public/data-010.csv', 'public/data-011.csv', 'public/data-012.csv', 'public/data-013.csv', 'public/data-014.csv', 'public/data-015.csv', 'public/data-016.csv', 'public/data-017.csv', 'public/data-018.csv', 'public/data-019.csv', 'public/data-020.csv', 'public/data-021.csv', 'public/data-022.csv', 'public/data-023.csv', 'public/data-024.csv', 'public/data-025.csv', 'public/data-026.csv', 'public/data-027.csv', 'public/data-028.csv', 'public/data-029.csv', 'public/data-030.csv', 'public/data-031.csv', 'public/data-032.csv', 'public/data-033.csv', 'public/data-034.csv', 'public/data-035.csv', 'public/data-036.csv', 'public/data-037.csv', 'public/data-038.csv', 'public/data-039.csv', 'public/data-040.csv', 'public/data-041.csv', 'public/data-042.csv', 'public/dat

UnboundLocalError: local variable 'X_all' referenced before assignment